In [6]:
import optuna
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier

In [2]:

data_train = pd.read_csv("./train.csv")
data_test = pd.read_csv('./test.csv')

In [3]:
data_train = data_train.drop(columns=["filename"])
data_test = data_test.drop(columns=["filename"])
data_train=data_train[data_train.kelas != "ABSZ"]
data_train=data_train[data_train.kelas != "TCSZ"]
data_train=data_train[data_train.kelas != "TNSZ"]
data_train=data_train[data_train.kelas != "SPSZ"]
data_train=data_train[data_train.kelas != "MYSZ"]
data_test=data_test[data_test.kelas != "ABSZ"]
data_test=data_test[data_test.kelas != "TCSZ"]
data_test=data_test[data_test.kelas != "TNSZ"]
data_test=data_test[data_test.kelas != "SPSZ"]
data_test=data_test[data_test.kelas != "MYSZ"]

In [4]:
classes_train = np.array(data_train['kelas'])
data_train = data_train.drop(columns=["kelas"])

In [12]:
classes_test = np.array(data_test['kelas'])
data_test = data_test.drop(columns=["kelas"])

# No Undersampling / Oversampling

In [10]:
def create_hyperparameter_space():
  space = {
      'n_estimators': optuna.quniform('n_estimators', 100, 1000, 50),
      'max_depth': optuna.quniform('max_depth', 2, 10, 1),
      'min_samples_split': optuna.quniform('min_samples_split', 2, 10, 1),
      'min_samples_leaf': optuna.quniform('min_samples_leaf', 1, 10, 1),
      'max_features': optuna.uniform('max_features', 0.1, 1.0),
  }
  return space

# Define the objective function
def objective(trial):
  # Create the random forest model
  model = RandomForestClassifier(
      n_estimators=int(trial.suggest_int('n_estimators', 100, 1000, 50)),
      max_depth=int(trial.suggest_int('max_depth', 2, 10, 1)),
      min_samples_split=int(trial.suggest_int('min_samples_split', 2, 10, 1)),
      min_samples_leaf=int(trial.suggest_int('min_samples_leaf', 1, 10, 1)),
      max_features=trial.suggest_uniform('max_features', 0.1, 1.0),
  )

  # Use 5-fold cross-validation to evaluate the model
  kfold = KFold(n_splits=5, shuffle=True)
  scores = cross_val_score(model, data_train , classes_train, cv=kfold)

  # Return the mean cross-validation score as the objective value
  return np.mean(scores)
study = optuna.create_study(direction='maximize')

# Run the optimization
study.optimize(objective, n_trials=100)

# Print the best hyperparameters
print(study.best_params)

[I 2023-01-03 15:13:42,283] A new study created in memory with name: no-name-0804aa89-0266-48ac-be8d-6b211b0d1f42
C:\Users\Alvin Buana\AppData\Local\Temp\ipykernel_12996\2656425140.py:19: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  max_features=trial.suggest_uniform('max_features', 0.1, 1.0),
[I 2023-01-03 15:13:46,405] Trial 0 finished with value: 0.6315151515151515 and parameters: {'n_estimators': 400, 'max_depth': 2, 'min_samples_split': 6, 'min_samples_leaf': 9, 'max_features': 0.477447706703786}. Best is trial 0 with value: 0.6315151515151515.
C:\Users\Alvin Buana\AppData\Local\Temp\ipykernel_12996\2656425140.py:19: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest

{'n_estimators': 500, 'max_depth': 2, 'min_samples_split': 7, 'min_samples_leaf': 8, 'max_features': 0.7261303336265162}


In [11]:
classifier = RandomForestClassifier(**study.best_params,random_state=0)
kfold = KFold(n_splits=5, shuffle=True)
# classifier.fit(data_train,classes_train)
scores = cross_val_score(classifier, data_train, classes_train, cv=kfold)
scores

array([0.69090909, 0.61818182, 0.68484848, 0.63636364, 0.52727273])

In [13]:
classifier.fit(data_train, classes_train)
test_score = classifier.predict(data_test)
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(test_score, classes_test)
print(accuracy)

0.6346749226006192


In [17]:
result = pd.DataFrame([np.concatenate((scores,[accuracy]))],columns=['K1(Train)','K2(Train)',
'K3(Train)','K4(Train)','K5(Train)','TEST'],index = ['Rft'])
result

,K1(Train),K2(Train),K3(Train),K4(Train),K5(Train),TEST
Rft,0.690909,0.618182,0.684848,0.636364,0.527273,0.634675
